In [1]:
from toy_model import *
from metrics import *
import wandb
import torch
import numpy as np

/opt/anaconda3/envs/toytrans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

wandb.login()

wandb: Currently logged in as: ce24b119 (jerrycloud3316-ai-club-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# to find kl loss between model and these processes
T0_proc1 = np.array([
    [0, 1, 0],
    [0, 0, 1], 
    [0, 0, 0.5]
])
T1_proc1 = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0.5, 0, 0]
])

# Different process
T0_proc2 = np.array([
    [0,1,0],
    [0,0,0],
    [0.5,0,0]
])
T1_proc2 = np.array([
    [0,0,0],
    [0,0,1],
    [0.5,0,0]
])
T0_proc3 = np.array([
    [0,1,0,0],
    [0,0,0,0.5],
    [0.5,0,0,0],
    [0,0,0,0.5]])
T1_proc3 = np.array([
    [0,0,0,0],
    [0,0,0.5,0],
    [0.5,0,0,0],
    [0.5,0,0,0]])   
process1 = MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1], seed=43)
process2 = MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc2, T1_proc2], seed=43)
process3 = MarkovData(n_gen=100, gen_len=32, n_states=4, d_vocab=2, T_list=[T0_proc3, T1_proc3], seed=43)

In [ ]:
dataset1=MarkovData(n_gen=5000, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1])
dataset2=MarkovData(n_gen=5000, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc2, T1_proc2])
merged_dataset = MergeMarkovDatasets(dataset1=dataset1, dataset2=dataset2,mixing_style='random')
for i in range (10):
    print(merged_dataset[i])

In [ ]:
x=torch.stack(merged_dataset.data)
onegramfreq, onegramcounts, onegramtotal=get_ngram_stats(x, n=1)
print(onegramfreq, onegramcounts, onegramtotal)
twogramfreq, twogramcounts, twogramtotal=get_ngram_stats(x, n=2)
print(twogramfreq, twogramcounts, twogramtotal)
threegramfreq, threegramcounts, threegramtotal=get_ngram_stats(x, n=3)
print(threegramfreq, threegramcounts, threegramtotal)


In [ ]:
dataset1=MarkovData(n_gen=50, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1], seed=43)
dataset2=MarkovData(n_gen=50, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc2, T1_proc2], seed=43)
test_dataset = MergeMarkovDatasets(dataset1=dataset1, dataset2=dataset2,mixing_style='random')

In [ ]:

metrics_config = MetricsConfig(
    track_markov_kl=True,
    markov_processes=[process1,process2,process3], 
    pos_start=6,
    
    track_ngrams=True,
    ngram_data=test_dataset,
    ngram_orders=[1, 2, 3],
    track_previous_token=True,
    track_in_context=False, 
    icl_data=test_dataset,
    icl_k1=5,
    icl_k2=32,
    track_composition=True,
    track_prefix_matching=False)

In [ ]:
model=load_model("proc1/seq_len32/X7/model300.pt","proc1/seq_len32/X7/model_cfg.pt")
print(model.cfg)

In [ ]:
model=load_model("proc1/seq_len32/X4/model300.pt","proc1/seq_len32/X4/model_cfg.pt")
print(model.cfg)

In [ ]:

model = train_model(
    dataset=merged_dataset,
    n_layers=2,
    d_model=16,
    n_heads=2, 
    attn_only=False,
    act_fn='silu',

    n_epochs=300,
    batch_size=64,
    lr=0.05,

    wandb=True,
    wandb_project_name="ICL",
    save_dir="proc1/seq_len32/X7*",
    save_every=20,
    print_every=10,


    metrics_config=metrics_config,
    metrics_log_interval=20
    )
wandb.finish()

O_2: training on original process for baseline

In [4]:
original_dataset=MarkovData(n_gen=10000, gen_len=32, n_states=4, d_vocab=2, T_list=[T0_proc3, T1_proc3])
for i in range(10):
    print(original_dataset[i])

{'tokens': tensor([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 1, 0, 0])}
{'tokens': tensor([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 0, 0, 0])}
{'tokens': tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 0, 0, 1, 0, 0, 0])}
{'tokens': tensor([1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 1])}
{'tokens': tensor([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 1, 0, 1, 0, 0, 1, 1])}
{'tokens': tensor([1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
        1, 1, 0, 1, 0, 0, 0, 1])}
{'tokens': tensor([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 0, 0, 0])}
{'tokens': tensor([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 0])}


In [5]:
test_dataset=MarkovData(n_gen=100, gen_len=32, n_states=4, d_vocab=2, T_list=[T0_proc3, T1_proc3], seed=43)

In [ ]:
x=torch.stack(original_dataset.data)
onegramfreq, onegramcounts, onegramtotal=get_ngram_stats(x, n=1)
print(onegramfreq, onegramcounts, onegramtotal)
twogramfreq, twogramcounts, twogramtotal=get_ngram_stats(x, n=2)
print(twogramfreq, twogramcounts, twogramtotal)
threegramfreq, threegramcounts, threegramtotal=get_ngram_stats(x, n=3)
print(threegramfreq, threegramcounts, threegramtotal)


In [6]:

metrics_config = MetricsConfig(
    track_markov_kl=True,
    markov_processes=[process1,process2,process3], 
    pos_start=5,
    
    track_ngrams=True,
    ngram_data=test_dataset,
    ngram_orders=[1, 2, 3],
    track_previous_token=True,
    track_in_context=False, 
    icl_data=test_dataset,
    icl_k1=5,
    icl_k2=32,
    track_composition=True,
    track_prefix_matching=False)

In [7]:
model = train_model(
    dataset=original_dataset,
    n_layers=2,
    d_model=16,
    n_heads=2, 
    attn_only=False,
    act_fn='silu',

    n_epochs=300,
    batch_size=64,
    lr=0.05,

    wandb=True,
    wandb_project_name="ICL",
    save_dir="proc1/seq_len32/o7*",
    save_every=20,
    print_every=10,


    metrics_config=metrics_config,
    metrics_log_interval=20
    )
wandb.finish()

Moving model to device:  cpu


  0%|          | 0/300 [00:00<?, ?it/s]

Metrics logged at step 50
Metrics logged at step 100


  0%|          | 1/300 [00:01<07:28,  1.50s/it]

Epoch 1 Validation Loss 0.6225011348724365
Metrics logged at step 150
Metrics logged at step 200


...

100%|█████████▉| 299/300 [08:44<00:01,  1.66s/it]

Epoch 299 Validation Loss 0.5246720910072327
Metrics logged at step 37400
Metrics logged at step 37450


100%|██████████| 300/300 [08:46<00:00,  1.76s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Metrics logged at step 37500
Epoch 300 Samples 8000 Step 124 Training Loss 0.5394248366355896
Epoch 300 Validation Loss 0.5310866236686707


1gram_kl_model_to_true,▁▁▁▁▁▁▁▁▁▂▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇█▇█▇███▇▇█
1gram_kl_true_to_model,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▆▆▇▇▇▇▇▇▇▇█
2gram_kl_model_to_true,▂▂▃▂▃▃▂▃▃▄▃▆▃▅▄▄▅▅▄▁▃▆▆▇▇▆▇▇▇▆▇██▇▇█████
2gram_kl_true_to_model,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▃▄▅▅▆▆▆▆▆▇▇███
3gram_kl_model_to_true,▃▃▃▂▃▂▂▃▃▂▁▄▄▄▂▃▂▄▄▃▅▅▄▇▆▆▅█▇█▇▇▆▆▇▇▇▇▇▇
3gram_kl_true_to_model,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇█
avg_k_composition_l0_l1,▂▁▁▁▁▁▂▂▃▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█████████▇▇▇▆
avg_prev_token_matching,▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▃▃▃▃▄▄▄▅▅▅▅▅▅▆▇▆▇▇████
avg_q_composition_l0_l1,▄▄▄▄▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▄▄▄▅▅▅▆▆▇███
avg_v_composition_l0_l1,███▇▇▇▇▇▆▆▅▅▄▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃
+28,...


In [ ]:
from numpy import dtype


def icl_kl(model, test_data, start_pos, end_pos):
    model.eval()
    x=torch.stack(test_data.data)  # (n_gen, gen_len)
    n_gen, gen_len=x.shape
    x_cond=x[:,:start_pos-1]  # (n_gen, start_pos)
    x_pred=x[:,start_pos:end_pos]  # (n_gen, end_pos-start_pos)
    kl_values=[]
    token_probs=[]

    print(test_data.states[0][0])
    for seq in range(n_gen):
        seq_probs=[]
        for t in range(gen_len):
            eta=test_data.states[seq][t]
            probs=test_data.model.token_probabilities(eta)
            seq_probs.append(probs)
        token_probs.append(seq_probs)
    token_probs=torch.tensor(token_probs,dtype=torch.float32)  # (n_gen, gen_len, d_vocab)
    with torch.no_grad():
        for p in range (start_pos, end_pos,1):
            logits=model(x[:,:p])  # (n_gen, p, d_vocab)
            #print(logits.shape)
            next_token_logit=logits[:, -1, :]
            next_token_prob=torch.softmax(next_token_logit, dim=-1)
            markov_next_token_prob = token_probs[:, p, :]
            #markov_next_token_prob=token_probs[:,p,:]
            #markov_next_token_prob=torch.tensor(markov_next_token_prob, dtype=torch.float32)
            next_token_prob_clamped=torch.clamp(next_token_prob, min=1e-8)
            markov_next_token_prob_clamped=torch.clamp(markov_next_token_prob, min=1e-8)
            kl_div=torch.sum(next_token_prob_clamped*torch.log(next_token_prob_clamped/markov_next_token_prob_clamped), dim=-1)
            kl_mean=kl_div.mean().item()
            kl_values.append(kl_mean)
            print(f"Position {start_pos+p}: KL divergence {kl_div.mean().item()}")
    return kl_values

In [ ]:
model=load_model("proc1/seq_len32/Z1/model300.pt","proc1/seq_len32/Z1/model_cfg.pt")
test_date=MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1], seed=43)
kl_vals= icl_kl(model=model,test_data= test_date,start_pos=2,end_pos=31)

In [ ]:
import matplotlib.pyplot as plt

kl_results = {}  # store {checkpoint: [kl_values across positions]}

for i in range(280, 300, 20):
    model = load_model(f"proc1/seq_len32/Z1/model{i}.pt",
                       f"proc1/seq_len32/Z1/model_cfg.pt")
    kl_vals = icl_kl(model=model, test_data=test_date, start_pos=2, end_pos=31)
    kl_results[i] = kl_vals  # list of KL losses per position

# Plotting
plt.figure(figsize=(40, 6))

positions = list(range(2, 31))  # token positions on x-axis

for ckpt, vals in kl_results.items():
    plt.plot(positions, vals, label=f"Model {ckpt}")

plt.xlabel("Token position")
plt.ylabel("KL divergence")
plt.title("KL divergence across positions for different model checkpoints")
plt.legend()
plt.grid(True)
plt.show()
    

In [ ]:
import matplotlib.pyplot as plt

kl_results = {}  # store {checkpoint: [kl_values across positions]}
model=load_model("proc1/seq_len32/X7*/model300.pt","proc1/seq_len32/X7*/model_cfg.pt")
test_date=MarkovData(n_gen=100, gen_len=32, n_states=4, d_vocab=2, T_list=[T0_proc3, T1_proc3], seed=43)
for i in range(280, 300, 20):
    model = load_model(f"proc1/seq_len32/Z1/model{i}.pt",
                       f"proc1/seq_len32/Z1/model_cfg.pt")
    kl_vals = icl_kl(model=model, test_data=test_date, start_pos=2, end_pos=31)
    kl_results[i] = kl_vals  # list of KL losses per position

# Plotting
plt.figure(figsize=(40, 6))

positions = list(range(2, 31))  # token positions on x-axis

for ckpt, vals in kl_results.items():
    plt.plot(positions, vals, label=f"Model {ckpt}")

plt.xlabel("Token position")
plt.ylabel("KL divergence")
plt.title("KL divergence across positions for different model checkpoints")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:


kl_results = {}  # store {checkpoint: [kl_values across positions]}
model_name="X7*"
test_data=MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1], seed=43)
for i in range(280, 310, 20):
    model = load_model(f"proc1/seq_len32/{model_name}/model{i}.pt",
                       f"proc1/seq_len32/{model_name}/model_cfg.pt")
    kl_vals = icl_kl(model=model, test_data=test_data, start_pos=2, end_pos=31)
    kl_results[i] = kl_vals  # list of KL losses per position

# Plotting
plt.figure(figsize=(40, 6))

positions = list(range(2, 31))  # token positions on x-axis

for ckpt, vals in kl_results.items():
    plt.plot(positions, vals, label=f"Model {ckpt}")

plt.xlabel("Token position")
plt.ylabel("KL divergence")
plt.title("KL divergence across positions for different model checkpoints")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:


kl_results = {}  # store {checkpoint: [kl_values across positions]}
model_name="X7*"
test_data=MarkovData(n_gen=100, gen_len=32, n_states=4, d_vocab=2, T_list=[T0_proc3, T1_proc3], seed=43)
for i in range(300, 310, 20):
    model = load_model(f"proc1/seq_len32/{model_name}/model{i}.pt",
                       f"proc1/seq_len32/{model_name}/model_cfg.pt")
    kl_vals = icl_kl(model=model, test_data=test_data, start_pos=2, end_pos=31)
    kl_results[i] = kl_vals  # list of KL losses per position

# Plotting
plt.figure(figsize=(40, 6))

positions = list(range(2, 31))  # token positions on x-axis

for ckpt, vals in kl_results.items():
    plt.plot(positions, vals, label=f"Model {ckpt}")

plt.xlabel("Token position")
plt.ylabel("KL divergence")
plt.title("KL divergence across positions for different model checkpoints")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

kl_results = {}  # store {checkpoint: [kl_values across positions]}
model_name="X7*_"
test_data=MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc2, T1_proc2], seed=43)
for i in range(300, 310, 20):
    model = load_model(f"proc1/seq_len32/{model_name}/model{i}.pt",
                       f"proc1/seq_len32/{model_name}/model_cfg.pt")
    kl_vals = icl_kl(model=model, test_data=test_data, start_pos=2, end_pos=31)
    kl_results[i] = kl_vals  # list of KL losses per position

# Plotting
plt.figure(figsize=(40, 6))

positions = list(range(2, 31))  # token positions on x-axis

for ckpt, vals in kl_results.items():
    plt.plot(positions, vals, label=f"Model {ckpt}")

plt.xlabel("Token position")
plt.ylabel("KL divergence")
plt.title("KL divergence across positions for different model checkpoints")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

kl_results = {}  # store {checkpoint: [kl_values across positions]}
model=load_model("proc1/seq_len32/X7*/model300.pt","proc1/seq_len32/X7*/model_cfg.pt")
test_date=MarkovData(n_gen=100, gen_len=32, n_states=3, d_vocab=2, T_list=[T0_proc1, T1_proc1], seed=43)
for i in range(300, 310, 20):
    model = load_model(f"proc1/seq_len32/Z1/model{i}.pt",
                       f"proc1/seq_len32/Z1/model_cfg.pt")
    kl_vals = icl_kl(model=model, test_data=test_date, start_pos=2, end_pos=31)
    kl_results[i] = kl_vals  # list of KL losses per position

# Plotting
plt.figure(figsize=(40, 6))

positions = list(range(2, 31))  # token positions on x-axis

for ckpt, vals in kl_results.items():
    plt.plot(positions, vals, label=f"Model {ckpt}")

plt.xlabel("Token position")
plt.ylabel("KL divergence")
plt.title("KL divergence across positions for different model checkpoints")
plt.legend()
plt.grid(True)
plt.show()